In [ ]:
def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file-tabs' + name, component_property='tabs'),
        Output('file-tabs' + name, component_property='children'),
        Input('uploaded-file' + name, component_property='data'),
        Input('file-tabs' + name, component_property='active_tab'),
    )
    def update_tabs(data, active_tab):
        try:
            # Handle empty data case
            if not data or not data[0]:
                return [], []
                
            encoded_files, file_contents, file_names = data
            
            # Basic validation
            if not isinstance(encoded_files, list) or not isinstance(file_names, list):
                return [], []
            
            if len(encoded_files) != len(file_names):
                return [], []
                
            # Create tabs list
            tabs = [
                {
                    'id': f'tab{i + 1}' + name,
                    'title': file_name
                }
                for i, file_name in enumerate(file_names)
            ]
            
            # Create children list with proper iframe handling
            children = []
            for i, (encoded_file, file_name) in enumerate(zip(encoded_files, file_names)):
                tab_id = f'tab{i + 1}' + name
                
                # Skip if encoded_file is empty or invalid
                if not encoded_file:
                    continue
                    
                # Handle different file types
                if file_name.lower().endswith('.pdf'):
                    # For PDFs, ensure we have proper data URI
                    if not encoded_file.startswith('data:application/pdf;base64,'):
                        # Fix missing header if needed
                        if encoded_file.startswith('data:base64,'):
                            encoded_file = encoded_file.replace('data:base64,', 'data:application/pdf;base64,')
                        elif ';base64,' in encoded_file:
                            encoded_file = 'data:application/pdf;base64,' + encoded_file.split(';base64,')[1]
                
                # Create iframe with improved styling and error handling
                iframe = html.Iframe(
                    id=f'output-data-upload-{tab_id}' + name,
                    src=encoded_file,
                    style={
                        'width': '100%',
                        'height': '1000px',
                        'border': 'none',
                        'overflow': 'auto'
                    }
                )
                
                children.append(iframe)
            
            # Validate we have matching tabs and children
            if len(tabs) != len(children):
                # Log error but return what we have
                print(f"Warning: Mismatch between tabs ({len(tabs)}) and children ({len(children)})")
                return tabs[:len(children)], children
                
            return tabs, children
            
        except Exception as e:
            print(f"Error in update_tabs: {str(e)}")
            # Return empty lists on error to maintain interface
            return [], []